# Twitter sentiment identification (Part -1)

In [35]:
# Importing required libraries
# To preprocess the data we need Tokenizer: here I have used nltk Tweet Tokenizer.
# nltk is used for natural language processing and its Tweet tokenizer works specially on tweets data
import nltk
from nltk.tokenize import TweetTokenizer
#from keras.preprocessing.text import Tokenizer
# imported Sequential model to built layer-by-layer model. 
from keras.models import Sequential
# imported Dense layer to build fully connected feedback network                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         to built layer-by-layer model. 
from keras.layers import Dense
import numpy as np #importing numpy
import pandas as pd #importing pandas
np.random.seed(7) 

In [36]:
# Step-1 Load data and tokenize it .
# load data by pandas library's read_csv function while changin the encoding of file to read floadt and string without any errors(which were present in original encodingof the file)
dataset2 = pd.read_csv("train_validation-1.csv", sep= ",",encoding = "ISO-8859-1")

# tokenizer divides a string into substrings by splitting on the specified character or string (such as white space)
# Tweettokenizer comes with few normalization features, strip_handles removes all the words starting with #symbol, @symbol and take out all the URLs
tokenizer = TweetTokenizer(strip_handles=True,reduce_len=True)
# All the tweets are stored in text variable
text = dataset2.iloc[:,2]
# here we are tokenizing it the tokenizer object we created earlier
tokens = text.apply(tokenizer.tokenize)
# check the data
tokens.head(5)


0             [is, so, sad, for, my, APL, friend, ...]
1            [I, missed, the, New, Moon, trailer, ...]
2                      [omg, its, already, 7:30, :, O]
3    [.., Omgaga, ., Im, sooo, im, gunna, CRy, ., I...
4    [i, think, mi, bf, is, cheating, on, me, !, !,...
Name: SentimentText, dtype: object

In [42]:
# Baseline

Baseline = (dataset2[dataset2['Sentiment']==1].shape[0] /dataset2.shape[0])*100
print("\nBaseline: %.2f%%"%(Baseline))


Baseline: 56.46%


In [43]:
## STEP-2 : Data Preprocessing 

# Import re to use regular expressions
import re
# convert to all the strings to lower case
for a,b in tokens.iteritems(): 
    tokens[a] =[i.lower() for i in b]
print(tokens[9]) 

# Substitute all the special characters and number's string with empty string,
for a,b in tokens.iteritems(): 
    tokens[a] =[re.sub(r'[^a-zA-z0-9\s]','',i) for i in b]
print(tokens[9])   


#Remove special characters like empty string and standalone alphanumeric characters
for a,b in tokens.iteritems(): 
    tokens[a] =[i for i in b if (i.isalpha())]
tokens[9]

['hmmm', '...', 'i', 'wonder', 'how', 'she', 'my', 'number', '@', '-', ')']
['hmmm', '', 'i', 'wonder', 'how', 'she', 'my', 'number', '', '', '']


['hmmm', 'i', 'wonder', 'how', 'she', 'my', 'number']

In [44]:
# Remove Stop Words like is, am , the , how , i etc.
# nltk.download('stopwords') # use it when running for the first tome

# Import stopwords list from nltk corpus module
from nltk.corpus import stopwords
# created unordered set of stopwords of english language
stop_words = set(stopwords.words('english'))
allow_words = ["not", "no"]
# match all the words in list of list with stop_words to remove them in final output
for i,y in tokens.iteritems(): 
    tokens[i] = [x for x in y if x not in stop_words or x in allow_words ]
        
tokens[9]
#reformed = [appos[word] if word in appos else word for word in words]

['hmmm', 'wonder', 'number']

In [45]:
# STEP - 3: Data Normalization
# Lemmatization
# nltk.download('wordnet') # use it when running for the first tome

# We lemmatize the stings to only root words because in sentiment analysis root word matter most and "Beautifully" and "Beautiful" is only one word for it. 
#Import WordNet lemmatizer 
from nltk.stem import WordNetLemmatizer
#create WordNetLemmatizer object
lemmat = WordNetLemmatizer()
for i,lst in tokens.iteritems(): 
     tokens[i] = [lemmat.lemmatize(word) for word in lst]
#print(tokens[:100])

In [46]:
# Convert the format from list of lists to lists of strings by joining all the tokenized strings to single strings sentence again.
for i,lst in tokens.iteritems(): 
     tokens[i] = [' '.join(str(word) for word in lst) ]

# convert the data type to dataframe
df = pd.Series( (x[0] for x in tokens) )
# Labels given in the original dataset file for Supervised learning 
yaxis= dataset2.iloc[:,1]
yaxis.shape

(99989,)

In [47]:
# STEP -4: Feature extraction (Vectorization)
# CountVectorizer allows to build a vocabulary (i.e dictionary) of known words and to encode new documents using that vocabulary.
from sklearn.feature_extraction.text import CountVectorizer


# create the CountVectorizer object
vectorizer = CountVectorizer()
# tokenize and build vocab
# tokenize here and fit the input dataframe on CountVectorizer model here.

# In the vocabulary keys are the distinct words in the input dataframe and value 
# corresponding is the count of times each word appeared in that input dataframe
vectorizer.fit(df)
# 
#print(vectorizer.vocabulary_)
# vectorized the input dataframe based on CountVectorizer model (encoding)  
vector = vectorizer.transform(df)
# convert scipy matrix to numpy array
vec_array = vector.toarray()
# Shape tell us the input of the first layer of model
print(vector.shape)
print(type(vec_array))



(99989, 48350)
<class 'numpy.ndarray'>


In [48]:
# Split whole dataset into 80% training and 20% testing data. Here we using sklearn to the slpit and 
# train_test_split function takes input x-axis (which is vectorized feature) , y-axis (labels given in the question), 
# test_size to know conversion ratio of test data and random_state variable to make random number predictable and get almost same answer at every run. 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = vec_array[:80000],vec_array[80000:], yaxis[:80000],yaxis[80000:] 

In [49]:
# defining Sequential model instance 
#model.add(Dense(12,activation='relu'))

model = Sequential()
# first layer of the model with input size of number of columns of vectorized array and 20 is the number of neurons with 'relu' as activation function 
model.add(Dense(12,input_dim=len(vec_array[0]),activation='relu'))
# 2 more hidden layers of model with 'relu' as activation function activation 
model.add(Dense(12,activation='relu'))
model.add(Dense(12,activation='relu'))
model.add(Dense(12,activation='relu'))
# Output layer of model with only 1 column for output is only one of the two (1,0) and activation function is sigmoid
model.add(Dense(1,activation='sigmoid'))

In [50]:
#compile the model using binary_crossentropy for loss because it a binary classification and adam as optimizer with accuracy as metrics
#optimizers = keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
# training the data by 5 epochs because accuracy changed very little after 3 iterations(epochs) and taking in batch size = 10 
model.fit(x_train,y_train,epochs=5,batch_size=15)

Epoch 1/5
80000/80000 [==============================] - 101s 1ms/step - loss: 0.5211 - acc: 0.7418
Epoch 2/5
80000/80000 [==============================] - 102s 1ms/step - loss: 0.4116 - acc: 0.8093
Epoch 3/5
80000/80000 [==============================] - 102s 1ms/step - loss: 0.3379 - acc: 0.8486
Epoch 4/5
80000/80000 [==============================] - 100s 1ms/step - loss: 0.2789 - acc: 0.8777
Epoch 5/5
80000/80000 [==============================] - 112s 1ms/step - loss: 0.2315 - acc: 0.9001


In [52]:
# Summarize data by evaluating and printing it accuracy on training data 
# Here accuracy is very high because model is trained with training set and we are evaluating accuracy of training set only.
scores = model.evaluate(x_train,y_train)
print("\n%s:%.2f%%"%(model.metrics_names[1],scores[1]*100))

80000/80000 [==============================] - 48s 595us/step

acc:92.94%


In [54]:
# Summarize data by evaluating and printing it accuracy on testing data 

scores = model.evaluate(x_test,y_test)
print("\n%s:%.2f%%"%(model.metrics_names[1],scores[1]*100))

19989/19989 [==============================] - 11s 548us/step

acc:73.49%
